In [1]:
import time
import sys
import json
import ee
import folium
import pandas as pd
import numpy as np
import math
import altair as alt
import urllib.request
from operator import itemgetter
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings('ignore')

import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

    
sys.path.insert(0, 'src')

from src.funcs import *
from src.cloudMask import *
from src.etl import *
    
# pip install ipygee
# import ipygee as ui

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWiAabfLOw5AMHTnSIUauuzEDikFqZEtHSlK1pbmP_Lhs3rcvAU0-oI

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# Defines date ranges for analysis
dateRange, epochs2022 = genDates()

observedMonths = dateRange[:35]     # [2019-01, 2021-11]
formattedAxisDates = [rangeFormatter(i[0]) for i in dateRange]

# Defines regions for analysis
madagascar, madagascarCenter, subset, subsetCenter = genEEGeometries()

# Loads feature collections
boundsLevel_1, parks = genFeatureCollections()

In [8]:
# Loads gif params
gifParams_1, gifParams_2, gifParams_3 = json.load(open('config/gifParams.json'))

gifParams_1["region"] = madagascar
gifParams_2["region"] = madagascar
gifParams_3["region"] = madagascar

In [6]:
# creates monthly gifs of NDVI, EVI in 2019 using Sentinel 2 images

sentinel = ee.ImageCollection("COPERNICUS/S2_SR"
            ).filterBounds(madagascar
            ).filterDate('2019-01-01', '2020-01-01')

# Dates for composite images in 2019
sentinelGifDates = [str(i).zfill(2) + "-" + "19" for i in range(1, 13)]

# compute monthly unmasked mosaic over Madagascar using ["B2", "B4", "B8"]
monthlyMosaic = ee.List.sequence(1, 12
                      ).map(lambda x: sentinel.filter(ee.Filter.calendarRange(x, x, 'month')
                                             ).select(["B2", "B4", "B8"]
                                             ).mosaic(
                                             ).set('month', x))


# use mosaic to compute an image with [NDVI, EVI] bands
monthlyVI = ee.ImageCollection(monthlyMosaic).map(lambda x: calcVI(x))

fp_1, fp_2 = "gifs/sentinelNDVI.gif", "gifs/sentinelEVI.gif"

downloadGif(monthlyVI.select("NDVI"), gifParams_1, fp_1, sentinelGifDates, ("80%", "90%"), 500)
downloadGif(monthlyVI.select("EVI"), gifParams_1, fp_2, sentinelGifDates, ("80%", "90%"), 500)

# View generated gifs
geemap.show_image("gifs/sentinelNDVI.gif")
geemap.show_image("gifs/sentinelEVI.gif")

Output()

Output()

In [203]:
# creates monthly gifs of NDVI, EVI 2019-2021 using MODIS vegetation indices

# Loads MODIS VI collection
modis = ee.ImageCollection("MODIS/006/MOD13Q1"
         ).filterBounds(madagascar
         ).filterDate("2019-01-01", "2021-12-01")

# Gets MODIS image dates for gif annotation 
modisMetadata = ee.List(modis.map(lambda x: x.metadata('system:time_start'))).getInfo()
modisGifDates = [i["properties"]["system:index"] for i in modisMetadata["features"]]

# Use derived VI to generate and save gif
fp_3, fp_4 = "gifs/modisNDVI.gif", "gifs/modisEVI.gif"
downloadGif(modis.select("NDVI"), gifParams_2, fp_3, modisGifDates, ("72.5%", "90%"), 250)
downloadGif(modis.select("EVI"), gifParams_2, fp_4, modisGifDates, ("72.5%", "90%"), 250)


# View generated gifs
geemap.show_image("gifs/modisNDVI.gif")
geemap.show_image("gifs/modisEVI.gif")

Output()

Output()

## Cloud Masking

In [8]:
# 2019 monthly sentinel composite images with cloud masking

for i in range(len(dateRange[:12])):
    startDate, endDate = dateRange[i][0], dateRange[i][1]
    
    # Masks images in a month and creates a composite image     
    s2_sr_collection = get_s2_sr_cld_col(subset, startDate, endDate)
    s2_sr_median = s2_sr_collection.map(add_cld_shdw_mask
                                ).map(apply_cld_shdw_mask
                                ).median()

    # Adds red, green, and blue bands to map
    m = folium.Map(location=subsetCenter, zoom_start=10)
    m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 
                 'min': 0, 'max': 2500, 'gamma': 1.1},
                 'S2 cloud-free mosaic')

    m.add_child(folium.LayerControl())
    print(startDate + " - " + endDate)
    display(m)
    print("\n \n")

2019-01-01 - 2019-02-01



 

2019-02-01 - 2019-03-01



 

2019-03-01 - 2019-04-01



 

2019-04-01 - 2019-05-01



 

2019-05-01 - 2019-06-01


...


 

2019-08-01 - 2019-09-01



 

2019-09-01 - 2019-10-01



 

2019-10-01 - 2019-11-01



 

2019-11-01 - 2019-12-01



 

2019-12-01 - 2020-01-01


In [9]:
# layer 1: NDVI with cloud masking
# layer 2: NDVI without cloud masking
# layer 3: normalized difference between layers 1, 2


for i in range(len(dateRange[:12])):
    startDate, endDate = dateRange[i][0], dateRange[i][1]

    s2_mask, s2_noMask = get_s2_Modified(subset, startDate, endDate)  
    maskedNdvi = s2_mask.map(add_cld_shdw_mask
                     ).map(apply_cld_shdw_mask
                     ).median(
                     ).normalizedDifference(["B8", "B4"])
    
    unmaskedNdvi = s2_noMask.median().normalizedDifference(["B8", "B4"])
    ndviDifference = (maskedNdvi.select("nd").subtract(unmaskedNdvi.select("nd"))).divide(maskedNdvi.select("nd").add(unmaskedNdvi.select("nd")))

    m = folium.Map(location=subsetCenter, zoom_start=10)
    m.add_ee_layer(maskedNdvi,
                {"bands": ["nd"],
                 "palette": ["white", "green"]
               }, "Masked NDVI")

    m.add_ee_layer(unmaskedNdvi,
                {"bands": ["nd"],
                 "palette": ["white", "green"]
               }, "Unmasked NDVI")

    m.add_ee_layer(ndviDifference,
                {"bands": ["nd"],
                 "min": -0.25, "max": 0.25,
                 "palette": ["red", "white", "green"]
               }, "Normalized NDVI Difference")

    m.add_child(folium.LayerControl())

    print(startDate + " - " + endDate)
    display(m)
    print("\n \n")

2019-01-01 - 2019-02-01



 

2019-02-01 - 2019-03-01



 

2019-03-01 - 2019-04-01



 

2019-04-01 - 2019-05-01



 

2019-05-01 - 2019-06-01


...


 

2019-08-01 - 2019-09-01



 

2019-09-01 - 2019-10-01



 

2019-10-01 - 2019-11-01



 

2019-11-01 - 2019-12-01



 

2019-12-01 - 2020-01-01


## Feature Collections/Spatial Summaries

In [5]:
##### ONLY RUN IF NECESSARY. Runtime is around 30 minutes

# startTime = time.time()
# applyReducer(madagascar, observedMonths, boundsLevel_1, parks)
# print("Runtime: {} minutes".format(np.round(((time.time()-startTime)/60), 3)))

Runtime: 28.416 minutes


In [5]:
adminStats = pd.read_csv("data/adminDistrictStats.csv")
parkStats = pd.read_csv("data/nationalParkStats.csv")

In [6]:
# Monthly mean NDVI by Level 1 administrative boundaries
alt.layer(alt.Chart(adminStats
            ).mark_line(
            ).encode(x=alt.X("Month:O", sort=adminStats["Month"].unique()),
                     y="Mean NDVI:Q",
                     color="Region:N"), 
          alt.Chart(adminStats
            ).mark_circle(size=45
            ).encode(x=alt.X("Month:O", sort=adminStats["Month"].unique()),
                     y="Mean NDVI:Q",
                     color="Region:N",
                     tooltip=["Month", "Region", "Mean NDVI"])
  ).properties(height=250, width=250
  ).facet(facet="Region:N", columns=4)

alt.FacetChart(...)

## Trend Analysis

In [7]:
def plotCharts(df_1, df_2, color, region):
  """
  Returns two charts where the first displays observed mean ndvi and fitted values.
  A vertical line at 11/21 seperates fitted values from predicted values.
  The second chart displays detrended mean NDVI by plotting a model's residuals.
  """
  plot_1 = alt.layer(alt.Chart(df_1).mark_line(color=color              # observed ndvi line chart
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y="Mean NDVI:Q"),
                     alt.Chart(df_1).mark_circle(size=50, color=color  # observed ndvi scatterplot
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y="Mean NDVI:Q",
                                            tooltip=["Month", "Region", "Mean NDVI"]),
                     alt.Chart(df_1).mark_line(color="red"              # fitted ndvi line chart
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Fitted Mean NDVI:Q", title="Mean NDVI")),
                     alt.Chart(df_1).mark_circle(size=50, color="red"   # fitted ndvi scatterplot
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Fitted Mean NDVI:Q", title="Mean NDVI"),
                                            tooltip=["Month", "Region", "Fitted Mean NDVI", "Residual"]),
                     alt.Chart(pd.DataFrame({'Month': ['11/21']})
                       ).mark_rule().encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates)),
                     alt.Chart(df_2).mark_line(color="red"              # predicted ndvi line chart
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Predicted Mean NDVI:Q", title="Mean NDVI")),
                     alt.Chart(df_2).mark_circle(size=50, color="red"  # predicted ndvi scatterplot
                                   ).encode(x=alt.X("Month:O",
                                                     sort=formattedAxisDates),
                                            y=alt.Y("Predicted Mean NDVI:Q", title="Mean NDVI"),
                                            tooltip=["Month", "Region", "Predicted Mean NDVI"]))
  
  plot_2 = alt.layer(alt.Chart(df_1).mark_line(color=color             # linechart of residuals/detrended NDVI
                                   ).encode(x=alt.X("Month:O",
                                                     sort=df_1["Month"].unique()),
                                            y="Residual:Q"),
                     alt.Chart(df_1).mark_circle(size=50, color=color  # scatterplot of residuals/detrended NDVI
                                   ).encode(x=alt.X("Month:O",
                                                     sort=df_1["Month"].unique()),
                                            y="Residual:Q",
                                            tooltip=["Month", "Region", "Residual"]))
  
  return alt.hconcat(plot_1.properties(title="{} Monthly Mean NDVI".format(region),
                                       height=250, width=400),
                     plot_2.properties(title="{} Detrended Monthly Mean NDVI".format(region),
                                       height=250, width=400))

In [11]:
# linear regression with subset of 7 administrative districts

charts_1, mseLinearMod = [], dict()
colors = ["#FFD13C", "#61ECB6", "#2ca02c", "#9467bd", "#17becf", "#D159F2", "#877138"]

for i in list(zip(adminStats["Region"].unique(), colors)):
    dfSubset = adminStats[adminStats["Region"] == i[0]]
    trainFeatures, target = dfSubset["Years Since Epoch (t)"].values.reshape(35,1), dfSubset["Mean NDVI"].values.reshape(35,1)
    predFeatures = np.reshape(epochs2022, (13, 1))

    linearMod = LinearRegression().fit(trainFeatures, target)
    dfSubset["Fitted Mean NDVI"] = np.round(linearMod.predict(trainFeatures), 3)
    dfSubset["Residual"] = np.round(target - linearMod.predict(trainFeatures), 3)
    mseLinearMod[i[0]] = np.mean(np.square(dfSubset["Residual"]))

    predDf = pd.DataFrame({"Month": formattedAxisDates[35:],
                           "Region": [i[0]] * 13,
                           "Predicted Mean NDVI": linearMod.predict(predFeatures).flatten()}).round(3)

    charts_1.append(plotCharts(df_1=dfSubset, df_2=predDf,
                               color=i[1], region=i[0]))

alt.vconcat(*charts_1)

alt.VConcatChart(...)

In [13]:
# harmonic models

charts_2, mseHarmonicMod = [], dict()

for i in list(zip(adminStats["Region"].unique(), colors)):
    dfSubset = adminStats[adminStats["Region"] == i[0]]
    timeRadiansTrain = dfSubset["Years Since Epoch (t)"].values * 2 * math.pi
    timeRadiansTest = np.array(epochs2022) * 2 * math.pi
    dfSubset[["cos", "sin"]] = [[math.cos(j), math.sin(j)] for j in timeRadiansTrain]

    trainFeatures, target = dfSubset[["Years Since Epoch (t)", "cos", "sin"]].values, dfSubset["Mean NDVI"].values.reshape(35, 1)

    predFeatures = []
    for j in epochs2022:
        timeRadians = j * 2 * math.pi
        predFeatures.append([j, math.cos(timeRadians), math.sin(timeRadians)])

    harmonicMod = LinearRegression().fit(trainFeatures, target)  
    dfSubset["Fitted Mean NDVI"] = np.round(harmonicMod.predict(trainFeatures), 3)
    dfSubset["Residual"] = np.round(target - harmonicMod.predict(trainFeatures), 3)
    mseHarmonicMod[i[0]] = np.mean(np.square(dfSubset["Residual"]))

    predDf = pd.DataFrame({"Month": formattedAxisDates[35:],
                           "Region": [i[0]] * 13,
                           "Predicted Mean NDVI": harmonicMod.predict(predFeatures
                                                            ).flatten()}
              ).round(3)

    charts_2.append(plotCharts(df_1=dfSubset, df_2=predDf, 
                               color=i[1], region=i[0]))

alt.vconcat(*charts_2)

alt.VConcatChart(...)

In [15]:
# AR(2) models

charts_3, mseAR = [], dict()

for i in list(zip(adminStats["Region"].unique(), colors)):
    dfSubset = adminStats[adminStats["Region"] == i[0]]

    # create lagged variable
    dfSubset["Mean NDVI (t-1)"] = dfSubset["Mean NDVI"].shift(1)
    dfSubset["Mean NDVI (t-2)"] = dfSubset["Mean NDVI"].shift(2)
    dfSubset = dfSubset.rename(columns={"Mean NDVI": "Mean NDVI (t)"}  # drop first two observations with NaN's
                      ).iloc[2:,].reset_index(drop=True)

    trainFeatures = dfSubset[["Mean NDVI (t-1)", "Mean NDVI (t-2)"]].values
    target = dfSubset["Mean NDVI (t)"].values.reshape(len(dfSubset), 1)

    ar_2 = LinearRegression().fit(trainFeatures, target)  # AR(2) model

    dfSubset["Fitted Mean NDVI"] = np.round(ar_2.predict(trainFeatures), 3)
    dfSubset["Residual"] = np.round(target - ar_2.predict(trainFeatures), 3)
    mseAR[i[0]] = np.mean(np.square(dfSubset["Residual"]))

    chart = alt.layer(alt.Chart(dfSubset).mark_line(color=i[1]
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Mean NDVI (t)"),
                      alt.Chart(dfSubset).mark_circle(color=i[1], size=50
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Mean NDVI (t)",
                                                 tooltip=["Month", "Region", "Mean NDVI (t)"]),
                      alt.Chart(dfSubset).mark_line(color="red"
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Fitted Mean NDVI"),
                      alt.Chart(dfSubset).mark_circle(color="red", size=50
                                        ).encode(x=alt.X("Month:O",
                                                          sort=formattedAxisDates),
                                                 y="Fitted Mean NDVI",
                                                 tooltip=["Month", "Region", "Fitted Mean NDVI", "Residual"]))

    charts_3.append(chart.properties(title="{} Fitted Monthly Mean NDVI".format(i[0]),
                                     height=300, width=500))

alt.vconcat(*charts_3)

alt.VConcatChart(...)

In [18]:
# MSE for training months
pd.DataFrame({"Regions": mseLinearMod.keys(),
              "Linear MSE": mseLinearMod.values(),
              "Harmonic MSE": mseHarmonicMod.values(),
              "AR(2) MSE": mseAR.values()}).round(0)

,Regions,Linear MSE,Harmonic MSE,AR(2) MSE
0,Alaotra Mangoro,423838.0,63667.0,218757.0
1,Amoron I Mania,805267.0,55461.0,137208.0
2,Analamanga,723590.0,48951.0,137122.0
3,Analanjirofo,556198.0,448775.0,599993.0
4,Androy,370415.0,121961.0,61322.0
5,Anosy,452298.0,27697.0,60752.0
6,Atsimo Andrefana,723893.0,83869.0,109396.0


In [40]:
# MODIS VI for raster visualization
# MODIS is used instead of Sentinel 2 because it doesn't have clouds and its 
# greater spatial resolution snd derived bands reduce runtime.
modis = ee.ImageCollection("MODIS/006/MOD13Q1"
         ).filterBounds(madagascar
         ).filterDate("2019-01-01", "2021-12-01"
         ).map(lambda x: x.addBands(x.metadata("system:time_start").divide(1e18).rename("t"))
         ).map(lambda x: x.addBands(ee.Image(1).rename("constant")))

In [41]:
# Fitted VI is calculated by fitting a robust linear model over all of Madagascar
# red represents the presence of vegetation

fittedNDVI, fittedEVI = ee.List([]), ee.List([])

for i in range(len(dateRange[:34])):   # 01/19 - 11/21
    start, end = dateRange[i]
    ndviBands = [['constant', 'time'], ["NDVI"]]
    eviBands = [['constant', 'time'], ["EVI"]]

    m = folium.Map(location=madagascarCenter, zoom_start=5.6)
    
    # Subset collection by month and create constant/time bands for every image
    modisSubset = modis.filterDate(start, end)
    
    # Selects bands and fit robust linear model     
    modisNDVI = modisSubset.map(lambda x: x.select(["constant", "t", "NDVI"]))
    modisEVI = modisSubset.map(lambda x: x.select(["constant", "t", "EVI"]))

    ndviMod = modisNDVI.reduce(ee.Reducer.robustLinearRegression(numX = 2, numY = 1))
    eviMod = modisEVI.reduce(ee.Reducer.robustLinearRegression(numX = 2, numY = 1))
    
    # Extract fitted VI and combine into single image
    fittedNDVI = fittedNDVI.add(ndviMod.select(['coefficients']
                                      ).arrayFlatten(ndviBands
                                      ).clip(madagascar))
    
    fittedEVI = fittedEVI.add(eviMod.select(['coefficients']
                                   ).arrayFlatten(eviBands
                                   ).clip(madagascar))
    

# Overlay administrative boundaries on image
regionBounds = ee.Image().byte().paint(featureCollection= boundsLevel_1, color=2, width= 2).visualize(palette='000000')

fittedNDVI = ee.ImageCollection(fittedNDVI).map(lambda x: x.blend(regionBounds.select("vis-red")))
fittedEVI = ee.ImageCollection(fittedEVI).map(lambda x: x.blend(regionBounds.select("vis-red")))


# Download and display gifs 
downloadGif(fittedNDVI.select("time_NDVI", "constant_NDVI"),
            gifParams_3, "gifs/modisFittedNDVI.gif", 
            [rangeFormatter(i[0]) for i in dateRange[:34]], ("80%", "90%"), 500)

downloadGif(fittedEVI.select("time_EVI", "constant_EVI"),
            gifParams_3, "gifs/modisFittedEVI.gif", 
            [rangeFormatter(i[0]) for i in dateRange[:34]], ("80%", "90%"), 500)

geemap.show_image("gifs/modisFittedNDVI.gif")
geemap.show_image("gifs/modisFittedEVI.gif")

Output()

Output()

In [42]:
# Creates a dicitionary with region names as keys and a list with its geometry/bound overlay as the value
regionBounds = {}

for i in boundsLevel_1.getInfo()["features"]:
    regionBounds[i["properties"]["ADM1_NAME"]] = [ee.Geometry(i["geometry"]),
                                                  ee.Image(
                                                   ).byte(
                                                   ).paint(featureCollection=ee.FeatureCollection(ee.Feature(ee.Geometry(i["geometry"]))),
                                                           color=2,
                                                           width=2
                                                   ).visualize(palette="000000")]

In [43]:
# Similar to the process above, but each region has its own fitted layer in the map 
# that is calculated by fitting a robust linear regression

for i in range(len(dateRange[:12])):  # 2019 only
    start, end = dateRange[i]
    bandNames = [['constant', 'time'], ["NDVI"]]
    m = folium.Map(location=madagascarCenter, zoom_start=5.6)
    
    for region, lst in regionBounds.items():     # returns a plot with each park as a layer
        geometry, bounds = lst
        collection = modis.filterBounds(geometry
                         ).filterDate(start, end
                         ).map(lambda x: x.select(["constant", "t", "NDVI"]))

        # each region has its own fitted model for every month
        robustLinearMod = collection.reduce(ee.Reducer.robustLinearRegression(numX = 2, numY = 1))
        # Overlay bounds on image and add as a layer
        robustFittedImage = robustLinearMod.select(['coefficients']
                                          ).arrayFlatten(bandNames
                                          ).clip(geometry
                                          ).blend(bounds.select("vis-red"))
        
        m.add_ee_layer(robustFittedImage, {"bands": ["time_NDVI", "constant_NDVI"], "gamma": 0.9},
                       "{} Fitted NDVI".format(region))

    m.add_child(folium.LayerControl())
    print(start + " - " + end)
    display(m)
    print("\n \n")

2019-01-01 - 2019-02-01



 

2019-02-01 - 2019-03-01



 

2019-03-01 - 2019-04-01



 

2019-04-01 - 2019-05-01



 

2019-05-01 - 2019-06-01


...


 

2019-08-01 - 2019-09-01



 

2019-09-01 - 2019-10-01



 

2019-10-01 - 2019-11-01



 

2019-11-01 - 2019-12-01



 

2019-12-01 - 2020-01-01


In [ ]:
# # choropleth

# for i in range(len(dateRange[:1])):  # 2019 only
#     start, end = dateRange[i]
#     bandNames = [['constant', 'time'], ["NDVI"]]
#     m = folium.Map(location=madagascarCenter, zoom_start=5.6)
    
#     for region, lst in regionBounds.items():     # returns a plot with each park as a layer
#         geometry, bounds = lst
#         image = modis.filterBounds(geometry
#                          ).filterDate(start, end
#                          ).map(lambda x: x.select(["constant", "t", "NDVI"])).mosaic()
#         linearRegression = image.reduceRegion(
#   reducer= ee.Reducer.linearRegression(numX=2, numY=1),
#   geometry: madagascar,
#   scale: 30,
# );

        
        
#         # each region has its own fitted model for every month
# #         robustLinearMod = collection.reduce(ee.Reducer.robustLinearRegression(numX = 2, numY = 1))
#         # Overlay bounds on image and add as a layer
# #         robustFittedImage = robustLinearMod.select(['coefficients']
# #                                           ).arrayFlatten(bandNames
# #                                           ).clip(geometry
# #                                           ).blend(bounds.select("vis-red"))
        
# #         m.add_ee_layer(robustFittedImage, {"bands": ["time_NDVI", "constant_NDVI"], "gamma": 0.9},
# #                        "{} Fitted NDVI".format(region))

#     m.add_child(folium.LayerControl())
#     print(start + " - " + end)
#     display(m)
#     print("\n \n")

### Appendix
extra unused code

In [ ]:
# # 2019 monthly NDVI, EVI
# monthlyVI = monthlyMosaic.map(lambda x: calcVI(ee.Image(x)))
# for i in range(1):
#   image = ee.Image(monthlyVI.get(i))
#   m = folium.Map(location=madagascarCenter, zoom_start=5.6)
#   m.add_ee_layer(image, {"bands": ["NDVI"], 
#                          "min": -0.2, "max": 1,
#                          "palette": ["white", "green"]},
#                  "NDVI")

#   m.add_ee_layer(image, {"bands": ["EVI"], 
#                          "min": -0.2, "max": 1,
#                          "palette": ["white", "green"]},
#                  "EVI")
  
#   m.add_child(folium.LayerControl())
  
#   print("{}/19".format(i+1) + "\n")
#   display(m)
#   print("\n")

In [187]:
# # 2019 monthly sentinel composite images with cloud masking
# maskedImages = ee.List([])
# for i in range(len(months2019)):
#   startDate, endDate = months2019[i][0], months2019[i][1]
#   s2_sr_collection = get_s2_sr_cld_col(subset, startDate, endDate)
#   maskedImages = maskedImages.add(s2_sr_collection.map(add_cld_shdw_mask
#                                 ).map(apply_cld_shdw_mask
#                                 ).median())

# # gifParams_3 = {
# #   'dimensions': 1000,
# #   'framesPerSecond': 1,
# #   "region": madagascar,
# #   "min": 0, "max": 2500,
# #   "gamma": 1.1
# # }

# # maskedURL = ee.ImageCollection(maskedImages).select("B4", "B3", "B2").getVideoThumbURL(gifParams_3)
# # urllib.request.urlretrieve(maskedURL, "gifs/madagascarMaskedImages_2019.gif")

In [ ]:
# 2019 monthly composite sentinel images with cloud masking and kernel reducer

# for i in range(len(months2019)):
#   startDate, endDate = months2019[i][0], months2019[i][1]
#   s2_sr_collection = get_s2_sr_cld_col(subset, startDate, endDate)
#   demo = s2_sr_collection.map(add_cld_shdw_mask
#                         ).map(apply_cld_shdw_mask
#                         ).map(lambda x: x.reduceNeighborhood(reducer = ee.Reducer.mean(),
#                                                              kernel = ee.Kernel.circle(3))
#                         ).median()

#   m = folium.Map(location=subsetCenter, zoom_start=10)

#   m.add_ee_layer(demo,
#                 {'bands': ['B4_mean', 'B3_mean', 'B2_mean'],
#                  "min": -100, "max": 2500, "gamma": 1.1},
#                 #  'gamma': [1.1, 1.3, 0.9]},
#                  'S2 cloud-free mosaic', True, 1, 9)

#   m.add_child(folium.LayerControl())
#   print(startDate + " - " + endDate)
#   display(m)
#   print("\n \n")

In [44]:
# bounds = boundsLevel_1.style(color='black',
#                              width=1,
#                              fillColor='109E5B',
#                              lineType='dotted',
#                              pointSize=6,
#                              pointShape='circle')